In [2]:
#import the modules
from gurobipy import *
import gurobipy as gp
from gurobipy import GRB
import os
import numpy as np
import pandas as pd
import csv 
import random
import sys
import matplotlib.pyplot as plt

## Call Once(clean the original data and write into a new csv file) <a name="call_once"></a>

In [3]:
def get_patient_improvement(data, drop_list):
    data = data.drop(drop_list, axis = 1)
    pat_num = data.shape[0]//2
    numneed = data.shape[1]
    improvements = {}
    for r in range(pat_num):        
        improvements[r] = [0 for i in range(numneed)]
        for c in range(numneed):
            improvements[r][c] =  data.iloc[2*r,c] - data.iloc[2*r+1,c] 
    improvements = pd.DataFrame.from_dict(improvements,orient='index')
    return improvements

def covert_to_int(string):
    [_, date, year] = string.split(",")
    [_,month,day] = date.split(" ")
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    if int(day)<10:
        day = "0" + day 
    for i in range(len(months)):
        if months[i] == month:
            if i >= 9:
                return year[1:] + "-" + str(i+1) + "-" + day
            else:
                return year[1:] + "-" + "0"+str(i+1) + "-"+day
            
def clean_data(df,treatment):
    #keep columns that we need
    index_to_keep = None
    if treatment == "ibhs":
        index_to_keep = [df.columns[2]] + list(df.columns[4:98]) + [df.columns[206]] + [df.columns[100]]
    else:
        index_to_keep = [df.columns[2]] + list(df.columns[4:98]) + [df.columns[198]] + [df.columns[100]]
    df = df[index_to_keep]
    
    #remove clients appear less than 2
    df = df.sort_values(by = 'client_id')
    if treatment == "ibhs":
        df = df[df['timeline_ibhs'] != 'CONTINUED']
    else:
        df = df[df['timeline_fbmh'] != 'CONTINUED']
    df = df[df.groupby('client_id')['client_id'].transform('count').ge(2)]
    
    # change the english strings to int strings
    rows = df.shape[0]
    for r in range(rows):
        df.iloc[r,96] = covert_to_int(df.iloc[r,96])
        if df.iloc[r,95] == 'INITIAL': 
            df.iloc[r,95] = 'A-INITIAL' #have initial before discharged for sorting later
            
    # sort data by client id, timeline(initial vs. discharge), and date of evaluation
    if treatment == "ibhs":
        df = df.sort_values(["client_id", "timeline_ibhs", "evaluated_at"], ascending=True)
        df.rename(columns = {'timeline_ibhs': 'timeline'}, inplace = True)
    else:
        df = df.sort_values(["client_id", "timeline_fbmh", "evaluated_at"], ascending=True)
        df.rename(columns = {'timeline_fbmh': 'timeline'}, inplace = True)
    df = df.reset_index(drop=True)
    
    # keep the earliest initial and latest discharged for each client, and remove any clients have both
    drop_rows = []
    prev_client = None
    cur_client = None
    next_client = None
    for r in range(df.shape[0]):
        cur_client = df.iloc[r, 0]
        if r < df.shape[0]-1:
            next_client = df.iloc[r+1, 0]
        else:
            next_client = None

        if df.iloc[r, 95] == 'A-INITIAL' and prev_client == cur_client and prev_client != None:
            drop_rows.append(r)
        elif df.iloc[r, 95] == 'DISCHARGED':
            if next_client == cur_client:
                drop_rows.append(r)
        prev_client = cur_client


    rows = df.index[drop_rows]
    df.drop(rows, inplace=True) 
    
    # remove clients with less than 2 rows
    df = df[df.groupby('client_id')['client_id'].transform('count').ge(2)]
    df = df.reset_index(drop=True)
    # df_ibhs.client_id.value_counts()
    
    
    #find list of patients
    clients = df['client_id'].unique().tolist()
    print("There are ",df.shape[0]//2," of patients for ", treatment, ".")
    
    return df, clients

def clean_all():
    ibhs = pd.read_excel('history_dataset/cans_ibhs_fbmh.xlsx',sheet_name=0)
    fbmh = pd.read_excel('history_dataset/cans_ibhs_fbmh.xlsx',sheet_name=1)
    fbmh_df,fbmh_patients = clean_data(fbmh,'fbmh')
    ibhs_df,ibhs_patients = clean_data(ibhs,'ibhs')
    return ((fbmh_df,fbmh_patients),(ibhs_df,ibhs_patients))


(fbmh_clean_df,fbmh_patients),(ibhs_clean_df,ibhs_patients) = clean_all()

#find an improvement table for every pateint(row) and need(col)
droplist = ['client_id','timeline', 'evaluated_at']
ibhs_imp = get_patient_improvement(ibhs_clean_df, droplist)
fbmhs_imp = get_patient_improvement(fbmh_clean_df, droplist)

#remove rows with all 0's
# df.loc[~(df==0).all(axis=1)]
ibhs_all_imp = ibhs_imp.loc[~(ibhs_imp==0).all(axis=1)].reset_index(drop=True)
fbmhs_all_imp = fbmhs_imp.loc[~(fbmhs_imp==0).all(axis=1)].reset_index(drop=True)

#write the improv_table to local
ibhs_all_imp.to_csv('improve_table_real_set/ibhs_all_improv_table.csv',index=False)
fbmhs_all_imp.to_csv('improve_table_real_set/fbmhs_all_improv_table.csv',index=False)
    

There are  2091  of patients for  fbmh .
There are  3838  of patients for  ibhs .


## Keep top-level items <a name="keep_top_level_items"></a>

In [3]:
old_ibhs_df = pd.read_csv('ibhs_all_improv_table.csv')
old_fbmh_df = pd.read_csv('fbmhs_all_improv_table.csv')


In [4]:
all_column_names = ['pp01', 'pp02', 'pp03', 'pp04', 
                    'pp05', 'pp06', 'pp07', 'pp08', 
                    'pp09', 'pp10', 'pp11', 'pp12', 
                    'pp13', 'pp14', 'pp15', 'pp16', 
                    'pp17', 'pp18', 'pp19', 'rb01', 
                    'rb02', 'rb03', 'rb04', 'rb05', 
                    'rb06', 'rb07', 'rb08', 'rb09', 
                    'rb10', 'rb11', 'rb12', 'rb13', 
                    'rb14', 'rb15', 'rb16', 'rb17', 
                    'rb18', 'rb19', 'rb20', 'rb21', 
                    'rb22', 'rb23', 'rb24', 'rb25', 
                    'rb26', 'rb27', 'rb28', 'fu01', 
                    'fu02', 'fu03', 'fu04', 'fu05', 
                    'fu06', 'fu07', 'fu08', 'fu09', 
                    'fu10', 'fu11', 'fu12', 'fu13', 
                    'fu14', 'fu15', 'fu16', 'fu17', 
                    'fu18', 'fu19', 'fu20', 'fu21', 
                    'fu22', 'fu23', 'fu24', 'fu25', 
                    'fu26', 'fu27', 'cs01', 'cn01', 
                    'cn02', 'cn03', 'cn04', 'cn05', 
                    'cn06', 'cn07', 'st01', 'st02', 
                    'st03', 'st04', 'st05', 'st06', 
                    'st07', 'st08', 'st09', 'st10', 'st11', 'st12']

In [5]:
# keep only 53 top-level items
# change index into column names
old_ibhs_df.columns = all_column_names
old_fbmh_df.columns = all_column_names
# keep the needed columns 
column_names_df = pd.read_csv('CANS Items for MILP Profile - Sheet1.csv')
column_names = list(column_names_df['Standradized Field'])
ibhs_df = old_ibhs_df[column_names]
fbmh_df = old_fbmh_df[column_names]
#change index back to integer 
index = [i for i in range(53)]
ibhs_df.columns = index
fbmh_df.columns = index

In [8]:
#write the top_level_improv_table to local
ibhs_df.to_csv('improve_table_real_set/ibhs_all_improv_table_top_level.csv',index=False)
fbmh_df.to_csv('improve_table_real_set/fbmhs_all_improv_table_top_level.csv',index=False)